# Inference Notebook for DigiLut Challenge

Two models trained are based on Dino using the Hugging face checkpoint "facebook/dinov2-small-imagenet1k-1-layer" and trained using the labeled data, from which patches of level 2 and level 3 according to openslide have been extracted. The models are trained using the gIoU of a patch with the ground truth bounding box as label.
Models were pretrained on a A100 GPU for less than a day. Inference is done on jupyterhub without GPU

## Install Libraries

Openslide is used to read the whole slide images and extract patches from them

In [3]:
pip install openslide-python


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Pytorch, Pytorch lightning and transformers are used to train and predict.

Stainlib is used for normalisation and image augmentation during training

In [4]:
!pip install pytorch-lightning
!pip install transformers
!pip install git+https://github.com/EIDOSLAB/torchstain.git
!pip install git+https://github.com/sebastianffx/stainlib.git


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
  Cloning https://github.com/EIDOSLAB/torchstain.git to /tmp/pip-req-build-wl5mbjq7
  Running command git clone --filter=blob:none --quiet https://github.com/EIDOSLAB/torchstain.git /tmp/pip-req-build-wl5mbjq7
  Resolved https://github.com/EIDOSLAB/torchstain.git to commit 3a3eb2c8af2662b4a001cf134f9a1bd24e0bdba7
  Preparing metadata (setup.py) ... done

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
  Cloning https://github.com/sebastianffx/stainlib.git to /tmp/pip-req-build-51xkuvpf
  Running command git clone --filter=blob:none --quiet https://github.com/sebastianffx/stainlib.git /tmp/pip-req-build-51xkuvpf
  Resolved https://github.com/sebastianffx/stainlib.git to commit cf0b60352404f60d2a797d04b343bad22013c

## Select appropriate level 3 patches

In this code, we consider 'levels' as openslide defines them. We first iter through the whole slide images with a defined step size and slelect the patches where the non-white and non-artefact content is above a certain threshold. We then save those patches in a csv file.

This step requires a compromise between performance and inference time.

If necessary, change the path of the images (now in folder "../data/images") and of the target patch for normalization

In [5]:
import sys
import os
sys.path.append('/home/jovyan')

from extract_level_3.src.create_level_3_csv import ExtractLevel3PatchesModule
#from extract_level_3.src.extract_relevant_patches import ExtractRelevantPatchesModule
patch_size=224

custom_patch_sizes = [
    0,
    0,
    0,
        224,
        210,
        200,
]

custom_step_sizes = [
    0,
    0,
    0,
        180,
        170,
        150,
]

extract_module_level_3 = ExtractLevel3PatchesModule(level=3, patch_size=patch_size, custom_step_sizes=custom_step_sizes,
                                              custom_patch_sizes=custom_patch_sizes,
                                              model_level_3_path="./model_level_3_train_soft_kl.ckpt",
                                              target_patch="target_patch_new.png",
                                              data_path="../data/images")

2024-08-14 05:04:07.765240: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-14 05:04:07.803466: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 AVX512_FP16 AVX_VNNI AMX_TILE AMX_INT8 AMX_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-14 05:04:08.515284: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


./model_level_3_train_soft_kl.ckpt
INIT
{'num_gpus': 1, 'seed': 0, 'log_dir': 'results', 'per_gpu_batchsize': 64, 'num_workers': 8, 'precision': '16-mixed', 'train_transform': ['train_transform2', 'norm', 'data/pKV3BMNUpm_a_15800_26000_224.png'], 'val_transform': ['val_transform', 'norm', 'data/pKV3BMNUpm_a_15800_26000_224.png'], 'image_size': 224, 'focal_loss_alpha': 0.05, 'focal_loss_gamma': 2.0, 'model_type': 'dino', 'vit': 'google/vit-large-patch16-224', 'swin': 'microsoft/swin-tiny-patch4-window7-224', 'swinv2': 'microsoft/swinv2-base-patch4-window12-192-22k', 'model_patch_size': 4, 'dino': 'facebook/dinov2-small-imagenet1k-1-layer', 'cls_loss_lambda': 1, 'hidden_size': 768, 'learning_rate': 1e-05, 'loss_exponent': 0.15, 'max_epoch': 20, 'pretrain': False, 'ckpt_path': '', 'hard': False, 'label_threshold': 0.01, 'finetune': False, 'images_dir': {'train': 'data/images/level_3', 'val': 'data/images/level_3'}, 'annotation_dir': {'train': 'data/annotations/level_3', 'val': 'data/annot

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-small-imagenet1k-1-layer and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([1]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LOADING
END LOADING


In [27]:
import pandas as pd

submission_template = ("../prediction/validate/submission/submission_sample.csv")
csv_file = ("patches_level_3_step_180.csv")

##### If necessary, change path of bbox count per image
file_name_counts_bb_path = "../prediction/validate/submission/filename_counts_bb.json"
############################

submission_df = pd.read_csv(submission_template)
validation_image_names = submission_df['filename'].unique()

In [6]:
from tqdm import tqdm

columns = ['prediction', 'x1',  'y1', 'x2',  'y2', 'filename', 'level', 'image_width', 'image_height']
df = pd.DataFrame(columns=columns)

results = []
for image_name in tqdm(validation_image_names):
    results.append(extract_module_level_3.classify_relevant_patches_image(image_name))
        
        # Retrieve results from processes
for result in results:
    result = pd.DataFrame(result)
    #print(result)
    df = df.append(result, ignore_index=True)
df = df.drop_duplicates()
df.to_csv(csv_file, index=False)
print(df.head())
print(df.shape)

  0%|          | 0/165 [00:00<?, ?it/s]

82944 197888


  1%|          | 1/165 [00:58<2:40:11, 58.61s/it]

82688 198144


  1%|          | 2/165 [01:39<2:10:30, 48.04s/it]

...

82944 197888


 99%|█████████▉| 164/165 [2:47:06<01:14, 74.01s/it]

82944 197888


100%|██████████| 165/165 [2:47:51<00:00, 61.04s/it]
/tmp/ipykernel_1621/2787106249.py:14: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(result, ignore_index=True)


  prediction   x1    y1   x2    y2          filename level image_width  \
0       None  564  4764  788  4988  hqi5y2OzZy_b.tif     3       10368   
1       None  600  4720  824  4944  hqi5y2OzZy_b.tif     3       10368   
2       None  644  4720  868  4944  hqi5y2OzZy_b.tif     3       10368   
3       None  600  4764  824  4988  hqi5y2OzZy_b.tif     3       10368   
4       None  644  4764  868  4988  hqi5y2OzZy_b.tif     3       10368   

  image_height  
0        24736  
1        24736  
2        24736  
3        24736  
4        24736  
(753231, 9)


## Predict lesions for level 3 patches

Using the csv file of relevant level 3 patches, we aim to predict which of those patches contain a lesion. To that end, we use a Dino model trained from the Hugging face checkpoint "facebook/dinov2-small-imagenet1k-1-layer". This model was trained on level 3 patches. We save in a csv file all patches which have a predicted lesion score higher than 0.4.

The inference time for this part is about 9 minutes on the Jupyter hub (no GPU) for an image which has 6400 relevant level 3 patches. In total, for all images the inference time takes several hours.
GPU could be used to speed up inference on another platform, in the standard syntax used by Pytorch.

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import pandas as pd
import openslide
from torchvision import transforms
from PIL import Image
from extract_level_3.src.binary_classifier import BinaryClassifier
from extract_level_3.src.utils import show_image_patch
from extract_level_3.src.utils import *

df = pd.read_csv('patches_level_3_step_180.csv')
predict_csv_file = ("predicted_patches_level_3_step_180.csv")

# Load your model (ensure you replace 'your_model.pth' with your actual model file)
model = BinaryClassifier.load_from_checkpoint("./model_level_3_train_soft_kl.ckpt")
model.eval()  # Set model to evaluation mode

# Custom Dataset class for patches
class PatchDataset(Dataset):
    def __init__(self, image_name, dataframe):
        self.image_path = os.path.join("../data/images", image_name)
        self.openslide_image = openslide.OpenSlide(self.image_path)
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        x, y, _, _ = self.dataframe[idx]
        patch = show_image_patch(
                    self.openslide_image,
                    x,
                    y,
                    224,
                    224,
                    3,
                )
        
        patch = patch.convert("RGB")
        _, _, patch = remove_artifacts(patch)
        preprocessed_patch = extract_module_level_3.patch_preprocessing(patch)
    
        
        return preprocessed_patch

df['prediction'] = None

# Group the DataFrame by image_path to process each image separately
grouped = df.groupby('filename')
if not os.path.exists(f"level_3_patches_step_180"):
    os.makedirs(f"level_3_patches_step_180")
# Iterate over each image
for image_name, group in tqdm(grouped):
    print(image_name)
    base_filename = image_name.split('.')[0]
    if os.path.exists(f"level_3_patches_step_180/{base_filename}_filtered_predictions.csv"):
        continue
    # Extract the coordinates for the current image
    coords = group[['x1', 'y1', 'image_width', 'image_height']]
    coords = coords.values
    # Create Dataset and DataLoader for the current image
    dataset = PatchDataset(image_name, coords)
    dataloader = DataLoader(dataset, batch_size=64, shuffle=False) 

    predictions = []

    # Make predictions for each batch of patches
    with torch.no_grad():  # Disable gradient calculation for inference
        for batch in tqdm(dataloader):
            batch = batch.squeeze(1)
            outputs = model(batch)
            probability = outputs.view(-1)
            probability = extract_module_level_3.sigmoid(probability)
            predictions.extend(probability.tolist())

    # Save or process predictions
    df.loc[group.index, 'prediction'] = predictions
    filtered_df = df[df['filename']==image_name]
    filtered_df = filtered_df[filtered_df['prediction'] > 0.4]
    base_filename = image_name.split('.')[0]
    safe_filename = f"level_3_patches_step_180/{base_filename}_filtered_predictions.csv"
    print("Saving to CSV")
    filtered_df.to_csv(safe_filename, index=False)
    print("Saved")
#df.to_csv(predict_csv_file, index=False)

./model_level_3_train_soft_kl.ckpt
INIT
{'num_gpus': 1, 'seed': 0, 'log_dir': 'results', 'per_gpu_batchsize': 64, 'num_workers': 8, 'precision': '16-mixed', 'train_transform': ['train_transform2', 'norm', 'data/pKV3BMNUpm_a_15800_26000_224.png'], 'val_transform': ['val_transform', 'norm', 'data/pKV3BMNUpm_a_15800_26000_224.png'], 'image_size': 224, 'focal_loss_alpha': 0.05, 'focal_loss_gamma': 2.0, 'model_type': 'dino', 'vit': 'google/vit-large-patch16-224', 'swin': 'microsoft/swin-tiny-patch4-window7-224', 'swinv2': 'microsoft/swinv2-base-patch4-window12-192-22k', 'model_patch_size': 4, 'dino': 'facebook/dinov2-small-imagenet1k-1-layer', 'cls_loss_lambda': 1, 'hidden_size': 768, 'learning_rate': 1e-05, 'loss_exponent': 0.15, 'max_epoch': 20, 'pretrain': False, 'ckpt_path': '', 'hard': False, 'label_threshold': 0.01, 'finetune': False, 'images_dir': {'train': 'data/images/level_3', 'val': 'data/images/level_3'}, 'annotation_dir': {'train': 'data/annotations/level_3', 'val': 'data/annot

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-small-imagenet1k-1-layer and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([1]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LOADING
END LOADING


  0%|          | 0/165 [00:00<?, ?it/s]

0Rv3MjnLWH_a.tif
0Rv3MjnLWH_b.tif
0X7KcW0X1w_a.tif
0X7KcW0X1w_b.tif
0Xl3mBHGYt_a.tif
17hsUcyTXf_b.tif
1xebGQuAM7_a.tif
1xebGQuAM7_b.tif
26cQuF2MVO_a.tif
2NSqVtgE4l_b.tif
2UbzKb19K0_a.tif
2UbzKb19K0_b.tif
2XE4wBhzed_a.tif
2h3k9I8BiG_a.tif
2kfmzlvPef_a.tif
30rDuvp5X0_a.tif
30rDuvp5X0_b.tif
42B8pnFQUm_b.tif
53y4oTNbsQ_a.tif
5n4N605nbw_a.tif
5n4N605nbw_b.tif
7HxL729fl6_a.tif
7NPTOwlzAp_a.tif
7YxmEi5lcF_a.tif
7YxmEi5lcF_b.tif
7kiGhyiFBZ_a.tif
7kiGhyiFBZ_b.tif
8xGdkL0vZt_a.tif
8xGdkL0vZt_b.tif
95xuz9H8aO_a.tif
9KcFSsPZec_b.tif
9NlPhYAFUH_a.tif
9NlPhYAFUH_b.tif
AIg925SQy8_a.tif
AIg925SQy8_b.tif
AnU5julYqS_a.tif
B25D2I1DpX_a.tif
BX9BSJROge_a.tif
BX9BSJROge_b.tif
CtVUXYvOjl_a.tif
DP3TH5qKPF_a.tif
E4XydHSrwt_b.tif
E9UNU8YPJP_a.tif
E9UNU8YPJP_b.tif
FSlCJqyahA_a.tif
FcY5A9PQM7_b.tif
G97Fh5Kl2L_a.tif
GJtN0vbkRH_a.tif
Gi47IqUeDF_b.tif
GvAgxdkkx5_a.tif
HcWxwcVwCe_a.tif
HdQZRfwGsv_b.tif
HwpLZ2WW9l_a.tif
HxNV8n8nF8_b.tif
IehPaaTZz6_a.tif
IoiXPtKMKv_b.tif
JRcVHxC7B0_b.tif
KXXg6YP4KC_a.tif
Kn5fOmiTn6_a.t

...

Saving to CSV
Saved
zGmWGNXqi1_a.tif



 99%|█████████▉| 164/165 [9:26:38<06:27, 387.03s/it]

Saving to CSV
Saved
zmvPBONP7H_a.tif



100%|██████████| 165/165 [9:30:03<00:00, 207.30s/it]

Saving to CSV
Saved


### Get level 3 bounding boxes, and from level 3 patches, get corresponding level 2 patches

From the predicted patches, we aim to create level 3 bounding boxes and extract level 2 patches from those bounding boxes.

First, we select all patches which have a prediction score higher than 0.6, and combine them to create bounding boxes when they orverlap.

When it is unsuficient, we select the next highest predicted patches, so that at there are at least n boundinb boxes, with n the number of ground truth bounding boxes to find per image.

Finally, we iter on those bounding boxes to select level 2 patches that are not white above a certain threshold.

In [4]:
def update_bbox(bbox1, bbox2):
    """Update bbox1 to cover both bbox1 and bbox2."""
    x_min = min(bbox1[0], bbox2[0])
    y_min = min(bbox1[1], bbox2[1])
    x_max = max(bbox1[2], bbox2[2])
    y_max = max(bbox1[3], bbox2[3])
    
    return (x_min, y_min, x_max, y_max)

def is_overlapping(bbox1, bbox2):
    """Check if two bounding boxes overlap."""
    return not (bbox1[2] < bbox2[0] or bbox1[0] > bbox2[2] or
                bbox1[3] < bbox2[1] or bbox1[1] > bbox2[3])

def find_n_non_overlapping_bboxes(df, n):
    """
    Find n non-overlapping bounding boxes with the highest prediction scores.
    
    Parameters:
    - df: Pandas DataFrame with columns ['x1', 'x2', 'y1', 'y2', 'prediction']
    - n: The number of non-overlapping bounding boxes to find
    
    Returns:
    - A list of selected bounding boxes and their scores
    """
    # Sort the dataframe by scores in descending order
    df_sorted = df.sort_values(by='prediction', ascending=False)
    
    selected_bboxes = []
    highest_score = df_sorted.iloc[0]['prediction']

    while not df_sorted.empty and (len(selected_bboxes) < n+2 or float(df_sorted.iloc[0]['prediction']) > 0.6):
        #if df_sorted.iloc[0]['prediction'] < 0.55:
        #    print("lower prediction",  df_sorted.iloc[0]['prediction'])
        # Take the bounding box with the highest score
        highest = df_sorted.iloc[0]
        highest_bbox = [(highest['x1'], highest['y1'], highest['x2'], highest['y2']), float(highest['prediction']), 0]

        df_sorted = df_sorted.iloc[1:]  # Remove the selected bbox from the list
        to_update = False
        for i in range(len(selected_bboxes)):
            bbox = selected_bboxes[i]
            if is_overlapping(bbox[0], highest_bbox[0]):
                to_update = True
            if is_overlapping(bbox[0], highest_bbox[0]) and highest_bbox[1] > 0.6:
                
                # If it overlaps with an already selected box, update the selected box
                new_bbox = update_bbox(bbox[0], highest_bbox[0])
                selected_bboxes[i][0] = new_bbox
                selected_bboxes[i][2] = bbox[2] + 1

                break
        if not to_update:
            # If it doesn't overlap with any selected box, add it to the selected list
            selected_bboxes.append(highest_bbox)
    
    return [box for box, pred, _ in selected_bboxes], [pred for box, pred, _ in selected_bboxes]

def level_2_bboxes(bboxes):
    bigger_bboxes = []
    
    for (xmin, ymin, xmax, ymax) in bboxes:
        xmin = int(xmin)*2
        xmax = int(xmax)*2
        ymin = int(ymin)*2
        ymax = int(ymax)*2
        
        bigger_bboxes.append((xmin, xmax, ymin, ymax))
    
    return bigger_bboxes

In [5]:
import pandas as pd
from tqdm import tqdm
from extract_level_3.src.utils import *

step_size_level_2 = 35
patch_size = 224
submission_template = ("../prediction/validate/submission/submission_sample.csv")
csv_file = (f"patches_level_3_step_180_level_2_step_{step_size_level_2}_version3.csv")
##### If necessary, change path of bbox count per image
file_name_counts_bb_path = "../prediction/validate/submission/filename_counts_bb.json"
submission_df = pd.read_csv(submission_template)
validation_image_names = submission_df['filename'].unique()

#all_predictions_df = pd.read_csv('all_predictions_level_3.csv')

#grouped = all_predictions_df.groupby('filename')
#print(grouped.head())
step_size_level_2 = int(step_size_level_2)
columns = ['prediction', 'x1',  'y1', 'x2',  'y2', 'filename', 'level', 'image_width', 'image_height']
patches_df = pd.DataFrame(columns=columns)
# Iterate over each image
patches = []
for image_name in tqdm(validation_image_names):
    base_filename = image_name.split('.')[0]
    file_path = f"level_3_patches_step_180/{base_filename}_filtered_predictions.csv"
    predictions_df = pd.read_csv(file_path)
    predictions_df = predictions_df.drop_duplicates()
    predictions_df = predictions_df[predictions_df.x1 != 'x1']
    slide = openslide.OpenSlide(os.path.join('../data/images', image_name))
    
    ## get the number of bounding boxes in the image from the submission csv file
    n = get_filename_count(file_name_counts_bb_path, image_name)
    selected_bboxes, pred = find_n_non_overlapping_bboxes(predictions_df, n)
    selected_bboxes = level_2_bboxes(selected_bboxes)
    #print(pred)
    image_width = int(predictions_df.iloc[0]['image_width'])*2
    image_height = int(predictions_df.iloc[0]['image_height'])*2
    #if image_name == '0X7KcW0X1w_a.tif':
    #    print(image_width, image_height)
    i= 0
    print(len(selected_bboxes), selected_bboxes)
    for bbox in selected_bboxes:
        tmp = 0
        xmin, xmax, ymin, ymax = bbox
        xmin, xmax, ymin, ymax = int(xmin), int(xmax), int(ymin), int(ymax)
        
        
        for x in range(xmin-step_size_level_2, xmax + step_size_level_2, step_size_level_2):
            for y in range(ymin-step_size_level_2, ymax + step_size_level_2, step_size_level_2):

                if x> 0 and y > 0 and x + patch_size <image_width -1 and y+ patch_size<image_height-1:
                    patch = show_image_patch(
                                            slide,
                                            x,
                                            y,
                                            224,
                                            224,
                                            2,
                                        )
                    _, _, no_artifact_patch = remove_artifacts(patch)
                    if not is_patch_white(no_artifact_patch, 0.20, 254):
                        patches.append({
                            'x1': x,
                            'x2': x + patch_size,
                            'y1': y,
                            'y2': y + patch_size,
                            'filename': image_name,
                            'level': 2,
                            'image_width': image_width*2,
                            'image_height': image_height*2,
                            'prediction': None
                        })
                            
                        if tmp==0:
                            tmp+=1
                            i+=1
                    #else:
    if i<n:
        print('skipped', i, n)


patches_df = patches_df.append(patches, ignore_index=True)
patches_df = patches_df.drop_duplicates()
patches_df.to_csv(csv_file, index=False)
print(patches_df.head())
print(patches_df.shape)
    


  0%|          | 0/165 [00:00<?, ?it/s]

The filename 'hqi5y2OzZy_b.tif' appears 48 times.
58 [(14488, 16496, 34280, 35176), (16280, 18088, 23160, 24536), (14760, 15296, 25288, 26096), (15688, 18896, 13648, 16136), (3960, 5696, 13200, 14848), (4048, 4888, 23248, 23928), (14328, 15688, 34280, 35696), (2328, 4136, 35840, 37488), (15960, 17336, 15240, 15688), (2760, 3688, 36000, 37576), (12840, 13736, 33440, 34408), (3960, 4408, 23928, 24376), (2400, 3296, 35840, 36536), (13040, 13736, 37040, 37576), (9960, 10496, 22280, 22728), (16640, 17248, 15240, 15688), (4280, 5248, 35640, 36808), (18360, 18896, 22800, 23608), (14760, 15528, 22280, 22888), (3960, 4408, 12840, 13576), (1560, 2096, 26240, 26848), (5160, 5696, 23160, 23696), (15240, 16048, 48912, 49448), (5840, 6808, 12680, 14096), (14760, 15296, 33440, 34048), (16280, 17088, 23160, 23696), (5840, 7128, 13200, 14096), (2848, 3888, 23480, 24448), (2488, 3296, 11480, 12376), (3080, 3976, 22440, 22888), (17248, 17696, 14760, 15208), (1560, 2096, 22280, 23088), (17640, 18896, 1404

  1%|          | 1/165 [01:22<3:45:50, 82.63s/it]

The filename '1xebGQuAM7_b.tif' appears 5 times.
7 [(5480, 5928, 15440, 15976), (13040, 13488, 27688, 28136), (7880, 8488, 3960, 4408), (15960, 16408, 2848, 3296), (13560, 14008, 14400, 14848), (6000, 6448, 7128, 7576), (6000, 6448, 4048, 4496)]


  1%|          | 2/165 [01:26<1:38:37, 36.30s/it]

...

The filename 'GJtN0vbkRH_a.tif' appears 1 times.
23 [(9960, 10728, 7648, 8488), (13560, 14096, 29928, 30536), (6840, 7648, 18840, 19576), (20176, 20712, 45240, 45976), (360, 1128, 30360, 30896), (8400, 9208, 28800, 29336), (7200, 8008, 44488, 45296), (8040, 8776, 10048, 10496), (19816, 20264, 33688, 34136), (19816, 20624, 6448, 7488), (8760, 9208, 17640, 18376), (7880, 8688, 8848, 9888), (5928, 6536, 45080, 45528), (13200, 13736, 32760, 33296), (19816, 20624, 19200, 19736), (13880, 14488, 5928, 6448), (7040, 7648, 30360, 30896), (14760, 15208, 3528, 3976), (1880, 2328, 8400, 8848), (9240, 9888, 8760, 9296), (16440, 16888, 36360, 36808), (680, 1128, 17480, 17928), (13128, 13576, 17928, 18376)]


 99%|█████████▉| 164/165 [25:49<00:09,  9.29s/it]

The filename 'efy8YfqxXZ_b.tif' appears 1 times.
3 [(5840, 6288, 40888, 41336), (14488, 14936, 9528, 9976), (13880, 14328, 38400, 38848)]


100%|██████████| 165/165 [25:51<00:00,  9.40s/it]
/tmp/ipykernel_560/787260766.py:80: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patches_df = patches_df.append(patches, ignore_index=True)


  prediction     x1     y1     x2     y2          filename level image_width  \
0       None  14453  34525  14677  34749  hqi5y2OzZy_b.tif     2       41472   
1       None  14453  34560  14677  34784  hqi5y2OzZy_b.tif     2       41472   
2       None  14453  34595  14677  34819  hqi5y2OzZy_b.tif     2       41472   
3       None  14453  34630  14677  34854  hqi5y2OzZy_b.tif     2       41472   
4       None  14488  34455  14712  34679  hqi5y2OzZy_b.tif     2       41472   

  image_height  
0        98944  
1        98944  
2        98944  
3        98944  
4        98944  
(544252, 9)


### Predict lesion probability for level 2 patches

Similar to the level 3 prediction: Using the csv file of relevant level 2 patches, we aim to predict which of those patches contain a lesion. To that end, we use a Dino model trained from the Hugging face checkpoint "facebook/dinov2-small-imagenet1k-1-layer". This model was trained on level 2 patches. We save in a csv file all patches which have a predicted lesion score higher than 0.4.

The inference time for this part is about 1 minutes 40 seconds on the Jupyter hub (no GPU) for an image which has 1280 relevant level 2 patches. In total, for all images the inference time takes several hours.
GPU could be used to speed up inference on another platform, in the standard syntax used by Pytorch.

In [6]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import pandas as pd
import openslide
from torchvision import transforms
from PIL import Image
from extract_level_3.src.binary_classifier import BinaryClassifier
from extract_level_3.src.utils import show_image_patch

step_size_level_2=35

csv_file = (f"patches_level_3_step_180_level_2_step_{step_size_level_2}_version3.csv")
predict_csv_file = (f"predicted_patches_level_2_steps_180_{step_size_level_2}.csv")

df = pd.read_csv(csv_file)
# Load your model (ensure you replace 'your_model.pth' with your actual model file)
model = BinaryClassifier.load_from_checkpoint("model_train_soft_kl_level2.ckpt")
model.eval()  # Set model to evaluation mode

# Custom Dataset class for patches
class PatchDataset(Dataset):
    def __init__(self, image_name, dataframe):
        self.image_path = os.path.join("../data/images", image_name)
        self.openslide_image = openslide.OpenSlide(self.image_path)
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        x, y, _, _ = self.dataframe[idx]
        patch = show_image_patch(
                    self.openslide_image,
                    x,
                    y,
                    224,
                    224,
                    2,
                )
        try:
        # Try to open the image and convert it to RGB
            patch = patch.convert("RGB")
            #_, _, patch = remove_artifacts(patch)

        except Exception as e:
            print(f"Error: {e}. Using a white image as fallback.")
            # If there's an error, create a white image
            patch = Image.new('RGB', (100, 100), color='white')  # Size (100, 100) is arbitrary

        preprocessed_patch = extract_module_level_3.patch_preprocessing(patch)
    
        
        return preprocessed_patch

df['prediction'] = None

# Group the DataFrame by image_path to process each image separately
grouped = df.groupby('filename')
#print(grouped.head())
if not os.path.exists(f"level_2_patches_180_{step_size_level_2}"):
    os.makedirs(f"level_2_patches_180_{step_size_level_2}")
# Iterate over each image
for image_name, group in tqdm(grouped):
    print(image_name)
    base_filename = image_name.split('.')[0]
    safe_filename = f"level_2_patches_180_{step_size_level_2}/{base_filename}_filtered_predictions.csv"
    #if os.path.exists(safe_filename):
    #    continue
    # Extract the coordinates for the current image
    coords = group[['x1', 'y1', 'image_width', 'image_height']]
    #print(coords.head())
    coords = coords.values
    # Create Dataset and DataLoader for the current image
    dataset = PatchDataset(image_name, coords)
    dataloader = DataLoader(dataset, batch_size=64, shuffle=False) 

    # Placeholder for predictions
    predictions = []

    i=0
    # Make predictions for each batch of patches
    with torch.no_grad():  # Disable gradient calculation for inference
        for batch in tqdm(dataloader):
            #print(batch.shape)
            batch = batch.squeeze(1)
            outputs = model(batch)
            probability = outputs.view(-1)
            probability = extract_module_level_3.sigmoid(probability)
            predictions.extend(probability.tolist())
            if i == 0:
                i+=1

    # Save or process predictions
    df.loc[group.index, 'prediction'] = predictions
    #print(df['prediction'].unique())
    #print(df.head())
    filtered_df = df[df['filename']==image_name]
    filtered_df = filtered_df[filtered_df['prediction'] > 0.4]
    base_filename = image_name.split('.')[0]

    safe_filename = f"level_2_patches_180_{step_size_level_2}/{base_filename}_filtered_predictions.csv"
    print("Saving to CSV")
    filtered_df.to_csv(safe_filename, index=False)
    print("Saved")

model_train_soft_kl_level2.ckpt
INIT
{'num_gpus': 1, 'seed': 0, 'log_dir': 'results', 'per_gpu_batchsize': 320, 'num_workers': 16, 'precision': '16-mixed', 'train_transform': ['train_transform2', 'norm', 'data/pKV3BMNUpm_a_15800_26000_224.png'], 'val_transform': ['val_transform', 'norm', 'data/pKV3BMNUpm_a_15800_26000_224.png'], 'image_size': 224, 'focal_loss_alpha': 0.05, 'focal_loss_gamma': 2.0, 'model_type': 'dino', 'vit': 'google/vit-large-patch16-224', 'swin': 'microsoft/swin-tiny-patch4-window7-224', 'swinv2': 'microsoft/swinv2-base-patch4-window12-192-22k', 'model_patch_size': 4, 'dino': 'facebook/dinov2-small-imagenet1k-1-layer', 'cls_loss_lambda': 1, 'hidden_size': 768, 'learning_rate': 0.0001, 'loss_exponent': 0.15, 'max_epoch': 20, 'pretrain': False, 'ckpt_path': '', 'hard': False, 'label_threshold': 0.01, 'finetune': False, 'images_dir': {'train': 'data/images/less_selective_patches_7_level_2_size_224_step_200', 'val': 'data/images/less_selective_patches_7_level_2_size_224_

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-small-imagenet1k-1-layer and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([1]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LOADING
END LOADING


  0%|          | 0/165 [00:00<?, ?it/s]

0Rv3MjnLWH_a.tif


...

Saving to CSV
Saved
zGmWGNXqi1_a.tif



 99%|█████████▉| 164/165 [11:01:34<05:00, 300.20s/it]

Saving to CSV
Saved
zmvPBONP7H_a.tif



100%|██████████| 165/165 [11:06:50<00:00, 242.49s/it]

Saving to CSV
Saved


## Get the bounding boxes from the level 2 patches

From the patches for each image that have the highest prediction of being a lesion, create bounding boxes

In [9]:
def update_bbox(bbox1, bbox2):
    """Update bbox1 to cover both bbox1 and bbox2."""
    x_min = min(bbox1[0],bbox2[0])
    y_min = min(bbox1[1],bbox2[1])
    x_max = max(bbox1[2],bbox2[2])
    y_max = max(bbox1[3],bbox2[3])
    
    return (x_min, y_min, x_max, y_max)

def is_overlapping(bbox1, bbox2):
    """Check if two bounding boxes overlap."""
    return not (bbox1[2] < bbox2[0] or bbox1[0] > bbox2[2] or
                bbox1[3] < bbox2[1] or bbox1[1] > bbox2[3])

def find_n_non_overlapping_bboxes_level_2(df, n):
    """
    Find n non-overlapping bounding boxes with the highest prediction scores.
    
    Parameters:
    - df: Pandas DataFrame with columns ['x1', 'x2', 'y1', 'y2', 'prediction']
    - n: The number of non-overlapping bounding boxes to find
    
    Returns:
    - A list of selected bounding boxes and their scores
    """
    # Sort the dataframe by scores in descending order
    df_sorted = df.sort_values(by='prediction', ascending=False)
    
    selected_bboxes = []
    highest_score = df_sorted.iloc[0]['prediction']
    while not df_sorted.empty and ((len(selected_bboxes) < n) or df_sorted.iloc[0]['prediction'] > 0.55): #higher seems to be better 
        if len(selected_bboxes) < n and df_sorted.iloc[0]['prediction'] < 0.55:
            print("lower prediction",  df_sorted.iloc[0]['prediction'])
        # Take the bounding box with the highest score
        highest = df_sorted.iloc[0]
        highest_bbox = [(highest['x1'], highest['y1'], highest['x2'], highest['y2']), highest['prediction'], 0]
        df_sorted = df_sorted.iloc[1:]  # Remove the selected bbox from the list
        
        to_update = False
        for i in range(len(selected_bboxes)):
            bbox = selected_bboxes[i]
            if is_overlapping(bbox[0], highest_bbox[0]):
                selected_bboxes[i][2] = bbox[2] + 1
                to_update = True
                if highest_bbox[1] > 0.55:#bbox[1]*0.8:
                # If it overlaps with an already selected box, update the selected box
                    new_bbox = update_bbox(bbox[0], highest_bbox[0])
                    selected_bboxes[i][0] = new_bbox
                to_update = True
                break
        if not to_update:
            # If it doesn't overlap with any selected box, add it to the selected list
            selected_bboxes.append(highest_bbox)
    
    return [box for box, _, _ in selected_bboxes], [pred for _, pred, _ in selected_bboxes], [num for _, _, num in selected_bboxes]

def reorder_bboxes(bboxes):
    resized_bboxes = []
    
    for (xmin, ymin, xmax, ymax) in bboxes:
        xmin = xmin*4
        xmax = xmax*4
        ymin = ymin*4
        ymax = ymax*4
        
        resized_bboxes.append((int(xmin), int(ymin), int(xmax), int(ymax)))
    
    return resized_bboxes

In [12]:
import pandas as pd
from tqdm import tqdm
from extract_level_3.src.utils import *


def update_csv_with_selected_boxes(df, csv_file, image_path, image_name, bounding_boxes, file_name_counts_bb_path):

      if image_name in df['filename'].unique():
            # Ensure only one image_name is processed
            ## get the number of bounding boxes in the image from the csv file

            count = get_filename_count(file_name_counts_bb_path, image_name)
            if len(bounding_boxes) < count:
                count = len(bounding_boxes)
                print(f"Not enough bounding boxes, only {count} available")
            if len(bounding_boxes) > count:
                bounding_boxes = bounding_boxes[:count]
            if count > 0:
                #bounding_boxes = bounding_boxes_df[['y1', 'y2', 'x1', 'x2']].head(count).values
                indices = df[df['filename'] == image_name].index
                for i, idx in enumerate(indices):
                    if i >= count:
                        break  
                    df.at[idx, 'x1'], df.at[idx, 'x2'], df.at[idx, 'y1'], df.at[idx, 'y2'] = bounding_boxes[i]

                #bounding_boxes = bounding_boxes[
                #bounding_boxes_df = bounding_boxes_df.iloc[count:]
            df.to_csv(csv_file, index=False)
      else:
            print('Image name is not in the csv submission file')

submission_template = ("../prediction/validate/submission/submission_sample.csv")
csv_file = (f"submission_stepsize_180_35_version_11.csv")
##### If necessary, change path of bbox count per image
file_name_counts_bb_path = "../prediction/validate/submission/filename_counts_bb.json"
submission_df = pd.read_csv(submission_template)
validation_image_names = submission_df['filename'].unique()


#grouped = all_predictions_df.groupby('filename')

for image_name in tqdm(validation_image_names):
    ## get the number of bounding boxes in the image from the csv file
    n = get_filename_count(file_name_counts_bb_path, image_name)
    ##########
    base_filename = image_name.split('.')[0]
    file_path = f"../extract_level_3/level_2_patches_180_35/{base_filename}_filtered_predictions.csv"
    predictions_df = pd.read_csv(file_path)
    predictions_df = predictions_df.nlargest(n*25, 'prediction')
    if len(predictions_df)<20*n: ##### if there is not enough level 2 patches
        file_path = f"../extract_level_3/level_3_patches_step_180/{base_filename}_filtered_predictions.csv"
        predictions_df = pd.read_csv(file_path)
        predictions_df["x1"] = predictions_df["x1"]*2
        predictions_df["y1"] = predictions_df["y1"]*2
        predictions_df["x2"] = predictions_df["x2"]*2
        predictions_df["y2"] = predictions_df["y2"]*2


    selected_bboxes, pred, num = find_n_non_overlapping_bboxes_level_2(predictions_df, n)
    #print(pred, num)    
    combined = list(zip(selected_bboxes, pred, num))
    combined_sorted = sorted(combined, key=lambda x: x[2], reverse=True)
    selected_bboxes, pred, num = zip(*combined_sorted)
    #print(pred, num)  
    selected_bboxes = reorder_bboxes(selected_bboxes)
    image_path = os.path.join("../data/images", image_name)
    #print(pred)
    update_csv_with_selected_boxes(submission_df, csv_file, image_path, image_name, selected_bboxes, file_name_counts_bb_path)

print('Done')
    

  1%|          | 1/165 [00:00<00:27,  5.87it/s]

The filename 'hqi5y2OzZy_b.tif' appears 48 times.
The filename 'hqi5y2OzZy_b.tif' appears 48 times.
The filename '1xebGQuAM7_b.tif' appears 5 times.
The filename '1xebGQuAM7_b.tif' appears 5 times.
The filename '8xGdkL0vZt_a.tif' appears 2 times.
The filename '8xGdkL0vZt_a.tif' appears 2 times.
The filename 'LQj5lC48hB_a.tif' appears 1 times.
The filename 'LQj5lC48hB_a.tif' appears 1 times.
The filename '9NlPhYAFUH_a.tif' appears 1 times.


  4%|▍         | 7/165 [00:00<00:06, 22.66it/s]

The filename '9NlPhYAFUH_a.tif' appears 1 times.
The filename '7YxmEi5lcF_a.tif' appears 10 times.
The filename '7YxmEi5lcF_a.tif' appears 10 times.
The filename 'QKwcTAZ3xm_a.tif' appears 29 times.
The filename 'QKwcTAZ3xm_a.tif' appears 29 times.
The filename 'M62FqXX2cW_a.tif' appears 14 times.
The filename 'M62FqXX2cW_a.tif' appears 14 times.
The filename 'BX9BSJROge_a.tif' appears 5 times.
lower prediction 0.5482991933822632
lower prediction 0.5442814826965332
lower prediction 0.5426422357559204
lower prediction 0.5403596758842468
lower prediction 0.5395274758338928
lower prediction 0.5382879376411438
lower prediction 0.5359520316123962
lower prediction 0.5346611142158508
lower prediction 0.5344206094741821
lower prediction 0.5338364839553833
lower prediction 0.5304074287414551
lower prediction 0.5285478234291077
lower prediction 0.5265113115310669
lower prediction 0.5231662392616272
lower prediction 0.5229663848876953
lower prediction 0.5223249793052673
lower prediction 0.5214410

  8%|▊         | 14/165 [00:00<00:05, 27.38it/s]

...

The filename 'FcY5A9PQM7_b.tif' appears 1 times.
The filename 'FcY5A9PQM7_b.tif' appears 1 times.
The filename 'gMYs85oTx2_a.tif' appears 1 times.
The filename 'gMYs85oTx2_a.tif' appears 1 times.
The filename 'ohz3unIR9h_a.tif' appears 2 times.
The filename 'ohz3unIR9h_a.tif' appears 2 times.
The filename '95xuz9H8aO_a.tif' appears 2 times.
The filename '95xuz9H8aO_a.tif' appears 2 times.
The filename 'xTNVJ85tMn_b.tif' appears 2 times.
The filename 'xTNVJ85tMn_b.tif' appears 2 times.
The filename '0Rv3MjnLWH_a.tif' appears 2 times.
The filename '0Rv3MjnLWH_a.tif' appears 2 times.
The filename 'M4QtF3H3Oe_a.tif' appears 2 times.
The filename 'M4QtF3H3Oe_a.tif' appears 2 times.
The filename 'w3DXwGBBdw_a.tif' appears 2 times.
The filename 'w3DXwGBBdw_a.tif' appears 2 times.
The filename '17hsUcyTXf_b.tif' appears 3 times.
The filename '17hsUcyTXf_b.tif' appears 3 times.
The filename 'hgF5qakk8A_b.tif' appears 2 times.
The filename 'hgF5qakk8A_b.tif' appears 2 times.
The filename 'i39wTB

 88%|████████▊ | 145/165 [00:02<00:00, 84.20it/s]

The filename 'KXXg6YP4KC_a.tif' appears 4 times.
The filename 'KXXg6YP4KC_a.tif' appears 4 times.
Not enough bounding boxes, only 3 available
The filename 'G97Fh5Kl2L_a.tif' appears 2 times.
The filename 'G97Fh5Kl2L_a.tif' appears 2 times.
The filename 'QxPvMHh6j8_b.tif' appears 2 times.
The filename 'QxPvMHh6j8_b.tif' appears 2 times.
The filename 'pfgu5Dsc2o_b.tif' appears 2 times.
The filename 'pfgu5Dsc2o_b.tif' appears 2 times.
The filename 'WipCgQtJPE_a.tif' appears 3 times.
The filename 'WipCgQtJPE_a.tif' appears 3 times.
The filename 'WLFisg4fXo_a.tif' appears 2 times.
The filename 'WLFisg4fXo_a.tif' appears 2 times.
The filename 'VjH7wZDQNM_b.tif' appears 3 times.
The filename 'VjH7wZDQNM_b.tif' appears 3 times.
The filename 'HcWxwcVwCe_a.tif' appears 2 times.
The filename 'HcWxwcVwCe_a.tif' appears 2 times.
The filename 'HwpLZ2WW9l_a.tif' appears 2 times.
The filename 'HwpLZ2WW9l_a.tif' appears 2 times.
The filename 'JRcVHxC7B0_b.tif' appears 3 times.
The filename 'JRcVHxC7B0_

100%|██████████| 165/165 [00:03<00:00, 54.43it/s]

The filename 'deXOH17EmS_a.tif' appears 3 times.
The filename 'eFtRfsPIFo_b.tif' appears 1 times.
The filename 'eFtRfsPIFo_b.tif' appears 1 times.
The filename '9NlPhYAFUH_b.tif' appears 1 times.
The filename '9NlPhYAFUH_b.tif' appears 1 times.
The filename 'yJrCf0Bme3_b.tif' appears 1 times.
The filename 'yJrCf0Bme3_b.tif' appears 1 times.
The filename 'rdcjsDq5X7_b.tif' appears 2 times.
lower prediction 0.5434280037879944
lower prediction 0.5389400124549866
lower prediction 0.5370506048202515
The filename 'rdcjsDq5X7_b.tif' appears 2 times.
The filename '7HxL729fl6_a.tif' appears 2 times.
The filename '7HxL729fl6_a.tif' appears 2 times.
The filename 'wzMDIiFoYH_a.tif' appears 1 times.
The filename 'wzMDIiFoYH_a.tif' appears 1 times.
The filename 'X0VlzvVmSs_a.tif' appears 1 times.
The filename 'X0VlzvVmSs_a.tif' appears 1 times.
The filename 'QgBvPeB8u5_a.tif' appears 1 times.
lower prediction 0.5387126207351685
The filename 'QgBvPeB8u5_a.tif' appears 1 times.
The filename '9KcFSsPZe

# How to run the code for a single image

To run this code, change the data path (data_path), the image name (image_name) and the number of requested bounding boxes (n)

Create csv with level 3 patches

In [34]:
import sys
import os
sys.path.append('/home/jovyan')
from tqdm import tqdm

from extract_level_3.src.create_level_3_csv import ExtractLevel3PatchesModule
#from extract_level_3.src.extract_relevant_patches import ExtractRelevantPatchesModule


#####TODO Input path to image
data_path="../data/images"
######TODO Input name of image (in .tif)
image_name = "GJtN0vbkRH_a.tif"
################# TODO 
n = 1 ##TODO input number of expected bounding boxes for the image
##############

patch_size=224
custom_patch_sizes = [
    0,
    0,
    0,
        224,
        210,
        200,
]
custom_step_sizes = [
    0,
    0,
    0,
        180,
        170,
        150,
]
extract_module_level_3 = ExtractLevel3PatchesModule(level=3, patch_size=patch_size, custom_step_sizes=custom_step_sizes,
                                              custom_patch_sizes=custom_patch_sizes,
                                              target_patch="target_patch_new.png",
                                              data_path=data_path)


columns = ['prediction', 'x1',  'y1', 'x2',  'y2', 'filename', 'level', 'image_width', 'image_height']
df = pd.DataFrame(columns=columns)

result = extract_module_level_3.classify_relevant_patches_image(image_name)
result = pd.DataFrame(result)

df = df.append(result, ignore_index=True)
df = df.drop_duplicates()
print(df.head())
print(df.shape)

82944 197888
  prediction    x1    y1    x2    y2          filename level image_width  \
0       None  6824  1164  7048  1388  GJtN0vbkRH_a.tif     3       10368   
1       None  6940  1120  7164  1344  GJtN0vbkRH_a.tif     3       10368   
2       None  7120  1120  7344  1344  GJtN0vbkRH_a.tif     3       10368   
3       None  6940  1164  7164  1388  GJtN0vbkRH_a.tif     3       10368   
4       None  7120  1164  7344  1388  GJtN0vbkRH_a.tif     3       10368   

  image_height  
0        24736  
1        24736  
2        24736  
3        24736  
4        24736  
(7426, 9)


/tmp/ipykernel_560/148828847.py:44: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(result, ignore_index=True)


Predict for those patches

In [35]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import pandas as pd
import openslide
from torchvision import transforms
from PIL import Image
from extract_level_3.src.binary_classifier import BinaryClassifier
from extract_level_3.src.utils import show_image_patch
from extract_level_3.src.utils import *


# Load your model (ensure you replace 'your_model.pth' with your actual model file)
model = BinaryClassifier.load_from_checkpoint("./model_level_3_train_soft_kl.ckpt")
model.eval()  # Set model to evaluation mode

# Custom Dataset class for patches
class PatchDataset(Dataset):
    def __init__(self, image_name, dataframe):
        self.image_path = os.path.join(data_path, image_name)
        self.openslide_image = openslide.OpenSlide(self.image_path)
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        x, y, _, _ = self.dataframe[idx]
        patch = show_image_patch(
                    self.openslide_image,
                    x,
                    y,
                    224,
                    224,
                    3,
                )
        
        patch = patch.convert("RGB")
        _, _, patch = remove_artifacts(patch)
        preprocessed_patch = extract_module_level_3.patch_preprocessing(patch)
    
        return preprocessed_patch

df['prediction'] = None

# Group the DataFrame by image_path to process the single image
grouped = df.groupby('filename')
group = grouped.get_group(image_name)

coords = group[['x1', 'y1', 'image_width', 'image_height']]
coords = coords.values
# Create Dataset and DataLoader for the current image
dataset = PatchDataset(image_name, coords)
dataloader = DataLoader(dataset, batch_size=64, shuffle=False) 

predictions = []

# Make predictions for each batch of patches
with torch.no_grad():  # Disable gradient calculation for inference
    for batch in tqdm(dataloader):
        batch = batch.squeeze(1)
        outputs = model(batch)
        probability = outputs.view(-1)
        probability = extract_module_level_3.sigmoid(probability)
        predictions.extend(probability.tolist())

df.loc[group.index, 'prediction'] = predictions
predictions_df = df[df['prediction'] > 0.4]
predictions_df = predictions_df.drop_duplicates()

./model_level_3_train_soft_kl.ckpt
INIT
{'num_gpus': 1, 'seed': 0, 'log_dir': 'results', 'per_gpu_batchsize': 64, 'num_workers': 8, 'precision': '16-mixed', 'train_transform': ['train_transform2', 'norm', 'data/pKV3BMNUpm_a_15800_26000_224.png'], 'val_transform': ['val_transform', 'norm', 'data/pKV3BMNUpm_a_15800_26000_224.png'], 'image_size': 224, 'focal_loss_alpha': 0.05, 'focal_loss_gamma': 2.0, 'model_type': 'dino', 'vit': 'google/vit-large-patch16-224', 'swin': 'microsoft/swin-tiny-patch4-window7-224', 'swinv2': 'microsoft/swinv2-base-patch4-window12-192-22k', 'model_patch_size': 4, 'dino': 'facebook/dinov2-small-imagenet1k-1-layer', 'cls_loss_lambda': 1, 'hidden_size': 768, 'learning_rate': 1e-05, 'loss_exponent': 0.15, 'max_epoch': 20, 'pretrain': False, 'ckpt_path': '', 'hard': False, 'label_threshold': 0.01, 'finetune': False, 'images_dir': {'train': 'data/images/level_3', 'val': 'data/images/level_3'}, 'annotation_dir': {'train': 'data/annotations/level_3', 'val': 'data/annot

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-small-imagenet1k-1-layer and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([1]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LOADING
END LOADING


100%|██████████| 117/117 [08:47<00:00,  4.51s/it]


Create csv of level 2 patches

In [36]:
def update_bbox(bbox1, bbox2):
    """Update bbox1 to cover both bbox1 and bbox2."""
    x_min = min(bbox1[0], bbox2[0])
    y_min = min(bbox1[1], bbox2[1])
    x_max = max(bbox1[2], bbox2[2])
    y_max = max(bbox1[3], bbox2[3])
    
    return (x_min, y_min, x_max, y_max)

def is_overlapping(bbox1, bbox2):
    """Check if two bounding boxes overlap."""
    return not (bbox1[2] < bbox2[0] or bbox1[0] > bbox2[2] or
                bbox1[3] < bbox2[1] or bbox1[1] > bbox2[3])

def find_n_non_overlapping_bboxes(df, n):
    """
    Find n non-overlapping bounding boxes with the highest prediction scores.
    """
    # Sort the dataframe by scores in descending order
    df_sorted = df.sort_values(by='prediction', ascending=False)
    
    selected_bboxes = []
    highest_score = df_sorted.iloc[0]['prediction']

    while not df_sorted.empty and (len(selected_bboxes) < n+2 or float(df_sorted.iloc[0]['prediction']) > 0.6):
        #if df_sorted.iloc[0]['prediction'] < 0.55:
        #    print("lower prediction",  df_sorted.iloc[0]['prediction'])
        # Take the bounding box with the highest score
        highest = df_sorted.iloc[0]
        highest_bbox = [(highest['x1'], highest['y1'], highest['x2'], highest['y2']), float(highest['prediction']), 0]

        df_sorted = df_sorted.iloc[1:]  # Remove the selected bbox from the list
        to_update = False
        for i in range(len(selected_bboxes)):
            bbox = selected_bboxes[i]
            if is_overlapping(bbox[0], highest_bbox[0]):
                to_update = True
            if is_overlapping(bbox[0], highest_bbox[0]) and highest_bbox[1] > 0.6:
                
                # If it overlaps with an already selected box, update the selected box
                new_bbox = update_bbox(bbox[0], highest_bbox[0])
                selected_bboxes[i][0] = new_bbox
                selected_bboxes[i][2] = bbox[2] + 1

                break
        if not to_update:
            # If it doesn't overlap with any selected box, add it to the selected list
            selected_bboxes.append(highest_bbox)
    
    return [box for box, pred, _ in selected_bboxes], [pred for box, pred, _ in selected_bboxes]

def level_2_bboxes(bboxes):
    bigger_bboxes = []
    
    for (xmin, ymin, xmax, ymax) in bboxes:
        xmin = int(xmin)*2
        xmax = int(xmax)*2
        ymin = int(ymin)*2
        ymax = int(ymax)*2
        
        bigger_bboxes.append((xmin, xmax, ymin, ymax))
    
    return bigger_bboxes

In [39]:
import pandas as pd
from tqdm import tqdm
from extract_level_3.src.utils import *

step_size_level_2 = 35
patch_size = 224

columns = ['prediction', 'x1',  'y1', 'x2',  'y2', 'filename', 'level', 'image_width', 'image_height']
patches_df = pd.DataFrame(columns=columns)
# Iterate over each image
patches = []
slide = openslide.OpenSlide(os.path.join('../data/images', image_name))
    


## get level 3 bounding boxes
selected_bboxes, pred = find_n_non_overlapping_bboxes(predictions_df, n)
selected_bboxes = level_2_bboxes(selected_bboxes)
image_width = int(predictions_df.iloc[0]['image_width'])*2
image_height = int(predictions_df.iloc[0]['image_height'])*2

for bbox in tqdm(selected_bboxes):
    xmin, xmax, ymin, ymax = bbox
    xmin, xmax, ymin, ymax = int(xmin), int(xmax), int(ymin), int(ymax)
    
    for x in range(xmin-step_size_level_2, xmax + step_size_level_2, step_size_level_2):
        for y in range(ymin-step_size_level_2, ymax + step_size_level_2, step_size_level_2):

            if x> 0 and y > 0 and x + patch_size <image_width -1 and y+ patch_size<image_height-1:
                patch = show_image_patch(
                                        slide,
                                        x,
                                        y,
                                        224,
                                        224,
                                        2,
                                    )
                _, _, no_artifact_patch = remove_artifacts(patch)
                if not is_patch_white(no_artifact_patch, 0.20, 254):
                    patches.append({
                        'x1': x,
                        'x2': x + patch_size,
                        'y1': y,
                        'y2': y + patch_size,
                        'filename': image_name,
                        'level': 2,
                        'image_width': image_width*2,
                        'image_height': image_height*2,
                        'prediction': None
                    })
                        

patches_df = patches_df.append(patches, ignore_index=True)
patches_df = patches_df.drop_duplicates()
print(patches_df.head())
print(patches_df.shape)

100%|██████████| 23/23 [00:17<00:00,  1.29it/s]

  prediction    x1    y1     x2    y2          filename level image_width  \
0       None  9925  7858  10149  8082  GJtN0vbkRH_a.tif     2       41472   
1       None  9925  7893  10149  8117  GJtN0vbkRH_a.tif     2       41472   
2       None  9925  7928  10149  8152  GJtN0vbkRH_a.tif     2       41472   
3       None  9925  7963  10149  8187  GJtN0vbkRH_a.tif     2       41472   
4       None  9925  7998  10149  8222  GJtN0vbkRH_a.tif     2       41472   

  image_height  
0        98944  
1        98944  
2        98944  
3        98944  
4        98944  
(7311, 9)



/tmp/ipykernel_560/3335892502.py:54: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  patches_df = patches_df.append(patches, ignore_index=True)


Get predicted lesion probability for level 2

In [40]:
import os
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import pandas as pd
import openslide
from torchvision import transforms
from PIL import Image
from extract_level_3.src.binary_classifier import BinaryClassifier
from extract_level_3.src.utils import show_image_patch


# Load your model (ensure you replace 'your_model.pth' with your actual model file)
model = BinaryClassifier.load_from_checkpoint("model_train_soft_kl_level2.ckpt")
model.eval()  # Set model to evaluation mode

# Custom Dataset class for patches
class PatchDataset(Dataset):
    def __init__(self, image_name, dataframe):
        self.image_path = os.path.join(data_path, image_name)
        self.openslide_image = openslide.OpenSlide(self.image_path)
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        x, y, _, _ = self.dataframe[idx]
        patch = show_image_patch(
                    self.openslide_image,
                    x,
                    y,
                    224,
                    224,
                    2,
                )

        patch = patch.convert("RGB")
        preprocessed_patch = extract_module_level_3.patch_preprocessing(patch)
    
        return preprocessed_patch

patches_df['prediction'] = None

# Group the DataFrame by image_path to process each image separately
grouped = patches_df.groupby('filename')
group = grouped.get_group(image_name)
coords = group[['x1', 'y1', 'image_width', 'image_height']]

coords = coords.values

# Create Dataset and DataLoader for the current image
dataset = PatchDataset(image_name, coords)
dataloader = DataLoader(dataset, batch_size=64, shuffle=False) 

# Placeholder for predictions
predictions = []

# Make predictions for each batch of patches
with torch.no_grad():  # Disable gradient calculation for inference
    for batch in tqdm(dataloader):
        #print(batch.shape)
        batch = batch.squeeze(1)
        outputs = model(batch)
        probability = outputs.view(-1)
        probability = extract_module_level_3.sigmoid(probability)
        predictions.extend(probability.tolist())


patches_df.loc[group.index, 'prediction'] = predictions
patches_df = patches_df[patches_df['prediction'] > 0.4]
print(patches_df.head())

model_train_soft_kl_level2.ckpt
INIT
{'num_gpus': 1, 'seed': 0, 'log_dir': 'results', 'per_gpu_batchsize': 320, 'num_workers': 16, 'precision': '16-mixed', 'train_transform': ['train_transform2', 'norm', 'data/pKV3BMNUpm_a_15800_26000_224.png'], 'val_transform': ['val_transform', 'norm', 'data/pKV3BMNUpm_a_15800_26000_224.png'], 'image_size': 224, 'focal_loss_alpha': 0.05, 'focal_loss_gamma': 2.0, 'model_type': 'dino', 'vit': 'google/vit-large-patch16-224', 'swin': 'microsoft/swin-tiny-patch4-window7-224', 'swinv2': 'microsoft/swinv2-base-patch4-window12-192-22k', 'model_patch_size': 4, 'dino': 'facebook/dinov2-small-imagenet1k-1-layer', 'cls_loss_lambda': 1, 'hidden_size': 768, 'learning_rate': 0.0001, 'loss_exponent': 0.15, 'max_epoch': 20, 'pretrain': False, 'ckpt_path': '', 'hard': False, 'label_threshold': 0.01, 'finetune': False, 'images_dir': {'train': 'data/images/less_selective_patches_7_level_2_size_224_step_200', 'val': 'data/images/less_selective_patches_7_level_2_size_224_

Some weights of Dinov2ForImageClassification were not initialized from the model checkpoint at facebook/dinov2-small-imagenet1k-1-layer and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([1]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


LOADING
END LOADING


100%|██████████| 115/115 [08:50<00:00,  4.61s/it]

   prediction    x1    y1     x2    y2          filename level image_width  \
13    0.46498  9925  8313  10149  8537  GJtN0vbkRH_a.tif     2       41472   
15   0.518905  9925  8383  10149  8607  GJtN0vbkRH_a.tif     2       41472   
16   0.415147  9925  8418  10149  8642  GJtN0vbkRH_a.tif     2       41472   
17   0.504609  9925  8453  10149  8677  GJtN0vbkRH_a.tif     2       41472   
26   0.466048  9960  7963  10184  8187  GJtN0vbkRH_a.tif     2       41472   

   image_height  
13        98944  
15        98944  
16        98944  
17        98944  
26        98944  


### Create bounding boxes from level 2 predictions

In [41]:
def find_n_non_overlapping_bboxes_level_2(df, n):
    """
    Find n non-overlapping bounding boxes with the highest prediction scores.
    """
    # Sort the dataframe by scores in descending order
    
    df_sorted = df.sort_values(by='prediction', ascending=False)
    
    selected_bboxes = []
    highest_score = df_sorted.iloc[0]['prediction']
    while not df_sorted.empty and ((len(selected_bboxes) < n) or df_sorted.iloc[0]['prediction'] > 0.55): #higher seems to be better 
        if len(selected_bboxes) < n and df_sorted.iloc[0]['prediction'] < 0.55:
            print("lower prediction",  df_sorted.iloc[0]['prediction'])
        # Take the bounding box with the highest score
        highest = df_sorted.iloc[0]
        highest_bbox = [(highest['x1'], highest['y1'], highest['x2'], highest['y2']), highest['prediction'], 0]
        df_sorted = df_sorted.iloc[1:]  # Remove the selected bbox from the list
        
        to_update = False
        for i in range(len(selected_bboxes)):
            bbox = selected_bboxes[i]
            if is_overlapping(bbox[0], highest_bbox[0]):
                selected_bboxes[i][2] = bbox[2] + 1
                to_update = True
                if highest_bbox[1] > 0.55:#bbox[1]*0.8:
                # If it overlaps with an already selected box, update the selected box
                    new_bbox = update_bbox(bbox[0], highest_bbox[0])
                    selected_bboxes[i][0] = new_bbox
                to_update = True
                break
        if not to_update:
            # If it doesn't overlap with any selected box, add it to the selected list
            selected_bboxes.append(highest_bbox)
    
    return [box for box, _, _ in selected_bboxes], [pred for _, pred, _ in selected_bboxes], [num for _, _, num in selected_bboxes]

def reorder_bboxes(bboxes):
    resized_bboxes = []
    
    for (xmin, ymin, xmax, ymax) in bboxes:
        xmin = xmin*4
        xmax = xmax*4
        ymin = ymin*4
        ymax = ymax*4
        
        resized_bboxes.append((int(xmin), int(ymin), int(xmax), int(ymax)))
    
    return resized_bboxes

In [48]:
import pandas as pd
from tqdm import tqdm
from extract_level_3.src.utils import *

    
if len(patches_df)<15*n: ##### if there is not enough level 2 patches
    predictions_df = predictions_df
    predictions_df["x1"] = predictions_df["x1"]*2
    predictions_df["y1"] = predictions_df["y1"]*2
    predictions_df["x2"] = predictions_df["x2"]*2
    predictions_df["y2"] = predictions_df["y2"]*2
else:
    predictions_df = patches_df
    predictions_df = predictions_df.nlargest(n*20, 'prediction')

selected_bboxes, pred, num = find_n_non_overlapping_bboxes_level_2(predictions_df, n)

## Sort the bounding boxes according to the number of overlapping patches present
combined = list(zip(selected_bboxes, pred, num))
combined_sorted = sorted(combined, key=lambda x: x[2], reverse=True)
selected_bboxes, pred, num = zip(*combined_sorted)

## Reorder and resize for level 0 bounding boxes
selected_bboxes = reorder_bboxes(selected_bboxes)[:n]

print("Result (x1, y1, x2, y2)", selected_bboxes)

Result (x1, y1, x2, y2) [(33880, 115340, 36596, 118056)]
